In [1]:
from collections import defaultdict
import ee
import pandas as pd
import geopandas as gpd
import geemap

In [2]:
ee.Authenticate()

Enter verification code: 4/1AZEOvhVVkRGU-fI0qtUMzJ-NrNv6-t2eQqtOGcrtdIuvT4_FLWJJOKmAksk

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
# Global warming potentials from IPCC AR5 Table 8.7. Using GWP-100.
# https://www.ipcc.ch/site/assets/uploads/2018/02/WG1AR5_Chapter08_FINAL.pdf
GWP = {
    'CO2': 1,
    'N2O': 265,
    'CH4': 28
}

In [4]:
ch4 = ee.Image('projects/wri-datalab/SCL-Cities/EDGAR_CH4_2020')
co2_exclshort = ee.Image('projects/wri-datalab/SCL-Cities/EDGAR_CO2exclshort_2020')
co2_short = ee.Image('projects/wri-datalab/SCL-Cities/EDGAR_CO2short_2020')
n20 = ee.Image('projects/wri-datalab/SCL-Cities/EDGAR_N2O_2020')
co2e = (co2_exclshort.add(co2_short).add(n20.multiply(GWP['N2O'])).add(ch4.multiply(GWP['CH4']))).multiply(31536) # avg tonnes per sq-m per year

In [5]:
urbext_2020 = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_2020')
urbext_data_2020 = geemap.ee_to_pandas(urbext_2020)

In [7]:
scale = 100

In [9]:
popcodes = worldpop_2020.aggregate_array('country').getInfo()

NameError: name 'worldpop_2020' is not defined

In [8]:
emi_res = defaultdict(list)
pop_res = defaultdict(list)
for i in range(len(urbext_data_2020)):
    ua = urbext_data_2020.iloc[i]
    ua_f = urbext_2020.filter(ee.Filter.eq('city_ids', str(ua['city_ids'])))
    geom = ua_f.geometry()
    if ua['country'] in popcodes:
        localpop_img = worldpop.filter(ee.Filter.equals('country', ua['country'])).select('population').first()
        localemi = co2e.multiply(co2e.pixelArea()).reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['b1']
        localpop = localpop_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['population']

        emi_res[ua['country']].append(localemi)
        pop_res[emi_res[ua['country']].append(localpop)]
    
    print(i, end=' ')
    
    
out_dicts = []
country_list = list(ato_res.keys())
country_list.sort()
for country in country_list:
    out_dicts.append(
        {
            'country': country,
            'urban_ghg_emissions_2020': sum(emi_res[country]),
            'urban_population_2020': sum(pop_res[country])
        }
    )
res_pd = pd.DataFrame(out_dicts)
res_pd.to_csv('CTY-4_GHGemissions.csv')

NameError: name 'popcodes' is not defined

In [38]:
emi_data

defaultdict(list,
            {'Australia': [('Perth', 7754445.862705403, 1162164.581867701),
              ('Darwin', 120872.19219847849, 15369.076132325063),
              ('Sunshine Coast', 263938.2463263016, 107030.7314644128),
              ('Newcastle/Lake Macquarie',
               991482.9028291972,
               202084.70036485983),
              ('Canberra', 469236.3391397819, 147402.415926059),
              ('Gold Coast', 1812628.618407423, 374841.8191746041),
              ('Adelaide', 6046880.91399612, 904612.7641329857),
              ('Brisbane', 9389769.938799009, 1455324.350231925),
              ('Melbourne', 27887011.014809787, 3008732.251532832),
              ('Sydney', 22964222.720980883, 3087117.540650502),
              ('Wollongong', 443773.8862676188, 114898.40638043555),
              ('Hobart', 310077.19898560207, 72703.79923859511),
              ('Townsville', 468117.96546124713, 122300.42972420591),
              ('Cairns', 343269.91415960673, 79706.280

In [39]:
results = {country: {
    'CO2e_tonnes_urbanonly': sum([i[1] for i in emi_data[country]]),
    'population_urbanonly': sum([i[2] for i in emi_data[country]]),
    'CO2e_tonnes_percapita_urbanonly': sum([i[1] for i in emi_data[country]]) / sum([i[2] for i in emi_data[country]])
} for country in emi_data}
result_df = pd.DataFrame(results).transpose()
result_df.to_csv('GHG_emissions_percapita.csv')